In [19]:
import json,re,os,sys,shutil
import concurrent.futures
import requests
from rich import console

url = '''https://www.chinabond.com.cn/cbiw/trs/getDocsByConditions'''
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}


def fetchResult(chnName,chnID,kw="",pg=1,acc=[]):
    r = requests.post(url
                      ,data={"childChnlName":chnName,"pageNum":pg,"pageSize":50,"parentChnlId":chnID
                            ,"keywords":kw}
                      ,headers=header)
    print(f"Fetch page={pg} with kw={kw}")
    assert r.status_code==200,f"failed on query: {r.text}"
    try:
        j = json.loads(r.text)
    except Exception as e:
        print(f"failed parse {r.text}")
    acc.extend(j['data']['data']['list'])
    if j['data']['data']['hasNextPage']:
        print("len of acc",len(acc))
        return fetchResult(childName,chnName,chnID,kw=kw,pg=pg+1,acc=acc)
    else:
        print(f"Ending with {j['data']['data']['hasNextPage']},return acc",len(acc))
        return acc

def download(z,target):
    _,pdfName,docName = z['appendixIds'].split("=")
    print(f"starting>> {docName}")
    url = z['DOCPUBURL'][:z['DOCPUBURL'].rfind("/")]+f"/{pdfName}"
    try:
        response = requests.get(url)
        open(os.path.join(target,docName), "wb").write(response.content)
        print(f"Done with {pdfName}")
    except Exception as e:
        print(e)
        print(f"Error downloading with {docName}")
    
    
def queryOnReports(kw,target):
    def consolResults(pg,acc=[]):
        r = requests.post(url
                          ,data={"childChnlName":"付息兑付与行权公告","pageNum":pg,"pageSize":50,"parentChnlId":1203
                                ,"keywords":kw}
                          ,headers=header)
        print(f"Fetch page={pg} with kw={kw}")
        assert r.status_code==200,f"failed on query: {r.text}"
        try:
            j = json.loads(r.text)
        except Exception as e:
            print(f"failed parse {r.text}")
        acc.extend(j['data']['data']['list'])
        if j['data']['data']['hasNextPage']:
            print("len of acc",len(acc))
            return consolResults(pg+1,acc)
        else:
            print(f"Ending with {j['data']['data']['hasNextPage']},return acc",len(acc))
            return acc
    result_items = consolResults(1,[])
    def isTrusteeReport(y):
        return True
    print(result_items)
    trustee_reports = filter(isTrusteeReport, result_items)
    filesDownloaded = open("downloaded.csv",'r')

    
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as exectuor:
        for f in trustee_reports:
            exectuor.submit(download, f)

    filesDownloaded.close()


In [13]:
rs = fetchResult("财务报告",1202,kw="吉时代")

Fetch page=1 with kw=吉时代
Ending with False,return acc 50


In [20]:
download(rs[0],".")

starting>> 吉时代2021年第三期个人汽车抵押贷款证券化信托2022年度审计报告.pdf
Done with P020230716693564055209.pdf


In [4]:
url = "https://www.chinabond.com.cn/cbiw/trs/getDocsByConditions"

r = requests.post(url
                  ,data={"childChnlName":"财务报告","pageNum":1,"pageSize":50,"parentChnlId":1202
                  ,"keywords":""}
                  ,headers=header)
json.loads(r.text)

{'success': True,
 'data': {'success': True,
  'data': {'total': 2505,
   'list': [{'AttachPic': 0,
     'FaXingRen': '476253',
     'DocTitle': '唯盈2021年第二期个人汽车抵押贷款证券化信托清算审计报告',
     'doccontent1': '',
     'docid': 853544613,
     'doctitle1': '唯盈2021年第二期个人汽车抵押贷款证券化信托清算审计报告',
     'FromId': '853544613',
     'OriginDocId': 853544613,
     'OriginMetaDataId': 853544613,
     'ChannelId': 1220,
     'AttachVideo': 0,
     'appendixIds': '1072422=P020230728811186842182.pdf=唯盈2021年第二期个人汽车抵押贷款证券化信托清算审计报告.pdf',
     'FromType': 0,
     'cNodeId': '10112122',
     'ShengXiaoShiJian': '2023-07-28 14:24:12',
     'WCMMetaTableChinaBandAllFieldsID': 651165,
     'DOCPUBURL': 'https://www.chinabond.com.cn/xxpl/ywzc_fxyfxdh/fxyfxdh_zqzl/zqzl_zjzzczj/zjzczq_ABS/ABS_cwbg/202307/t20230728_853544613.html',
     'MetaDataId': 853544613,
     'DocHtmlCon': '',
     'CrUser': 'zzpl',
     'DocRelTime': '2023-07-28 22:31:58',
     'ThumbFiles': '',
     'TopLevel': '0',
     'CrTime': '2023-07-28 22:31:5